In [1]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()

['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3']

In [ ]:
import pickle as pk
import tensorflow as tf
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"
for d in ['/device:GPU:2', '/device:GPU:3']:
  with tf.device(d):
    import keras
    from keras.backend.tensorflow_backend import set_session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    set_session(tf.Session(config=config))
    
    from keras.models import Sequential
    #from keras.layers import Dense, Dropout, Flatten
    from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose,Activation
    from keras.layers.normalization import BatchNormalization
    from keras import backend as K
    epochs = 10
    batch_size = 64
    pickle_train = open("../noiseBabble_20db.pickle","rb")
    x_train = pk.load(pickle_train)
    pickle_trainlabel = open("../cleanBabble.pickle","rb")
    y_train = pk.load(pickle_trainlabel)

    print("Got the input data")
    #x_train = np.asarray(x_train)
    y_train = np.asarray(y_train)
    print(np.asarray(x_train).shape)
    #     print(x_train[0:10].shape)
    X_train = []
    #X_train = np.asarray(X_train)
    for i in range(7,len(x_train)):
        X_train.append(x_train[i-7:i+1])
    #     temp = x_train[i-7]
    #     for j in range(1,8):
    #         temp = np.concatenate((temp,x_train[(i-7)+j]),axis=1)
    #     X_train.append(temp)

    X_train = np.asarray(X_train)
    print("H",np.asarray(X_train).shape)
    # X_train = X_train.reshape(X_train.shape[0], 129, 128, 1)
    # print("H2",np.asarray(X_train).shape)
    y_train = y_train.reshape(y_train.shape[0],1,129, 16)


    model = Sequential()
    model.add(Conv2D(64, kernel_size=(7, 7),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first",input_shape=(8,129,16)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(128, (5, 5),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256, (1, 1),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
    #model.add(Conv2DTranspose(128, (3, 3), activation='relu',padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2DTranspose(128, (3, 3),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2DTranspose(64, (5, 5),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2DTranspose(1, (7, 7),padding='valid',use_bias=True, kernel_initializer='glorot_uniform',bias_initializer='zeros',data_format= "channels_first"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.compile(loss=keras.losses.mean_squared_error,
                  optimizer=keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999),
                  metrics=['accuracy'])
    #model.build()
    #     def ATH():
    #         return 3.64*(pow(f/1000,0.8)) - 6.5*(np.exp(0.6*(pow(f/1000-3.3,2)))) + 0.001*(pow(f/1000,4))

    #     def myloss(y_true,y_pred,weights):
    #         return np.mean((weights(y_true-y_pred))**2)

    print(model.summary())
    with tf.device('/device:GPU:2') as open:
        model.fit(np.asarray(X_train), np.asarray(y_train[7:]),
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_split = 0.01
                 )

    model.save_weights('weights_8frames_10_final.h5')
    model.save('model_8frames_10_final.h5')